In [ ]:
#!pip install -q kaggle

In [ ]:
'''from google.colab import files
files.upload()'''

In [ ]:
'''!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!rm kaggle.json
!kaggle competitions download -c tuberculosis-drug-resistance-prediction-kan'''

  0% 0.00/91.0k [00:00<?, ?B/s]
100% 91.0k/91.0k [00:00<00:00, 28.7MB/s]
  0% 0.00/74.1k [00:00<?, ?B/s]
100% 74.1k/74.1k [00:00<00:00, 66.6MB/s]
  0% 0.00/443k [00:00<?, ?B/s]
100% 443k/443k [00:00<00:00, 62.7MB/s]
  0% 0.00/2.82k [00:00<?, ?B/s]
100% 2.82k/2.82k [00:00<00:00, 2.93MB/s]


In [ ]:
'''!unzip ./X_trainData_1.csv.zip
!rm ./X_trainData_1.csv.zip'''

Archive:  ./X_trainData_1.csv.zip
  inflating: X_trainData_1.csv       


In [ ]:
import pandas as pd
df = pd.read_csv('X_trainData_1.csv')
#df

In [ ]:
df1 = pd.read_csv('Y_trainData_1.csv')
#df1

In [ ]:
for i in df1.columns:
  print(df1[i].value_counts())

 1    2057
 0    1278
-1      58
Name: RIF, dtype: int64
 1    1832
 0    1524
-1      37
Name: INH, dtype: int64
 1    2246
 0     695
-1     452
Name: PZA, dtype: int64
 1    2346
 0     973
-1      74
Name: EMB, dtype: int64
-1    1312
 1    1061
 0    1020
Name: STR, dtype: int64
-1    2214
 1     942
 0     237
Name: CIP, dtype: int64
-1    2054
 1     752
 0     587
Name: CAP, dtype: int64
-1    2033
 1    1127
 0     233
Name: AMK, dtype: int64
-1    2056
 1    1070
 0     267
Name: MOXI, dtype: int64
-1    2703
 1     603
 0      87
Name: OFLX, dtype: int64
-1    2110
 1    1013
 0     270
Name: KAN, dtype: int64


In [ ]:
df = pd.read_csv('X_trainData_1.csv')
df1 = pd.read_csv('Y_trainData_1.csv')

df.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
df.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
df.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)
count=0
for ind in df1.index:
  x = df1['KAN'][ind]
  if x == -1:
    count+=1
    df.drop(index=ind,inplace=True)
    df1.drop(index=ind,inplace=True)
print(count)
df['KAN'] = df1['KAN']



2110


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit,RandomizedSearchCV,GridSearchCV
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import CategoricalNB
import numpy as np
import pickle

In [ ]:
ss = StratifiedShuffleSplit(n_splits=1,test_size=0.1,random_state=42)

traindata1 = df.drop(columns=['KAN'],axis=1)
y1 = df['KAN']

for train_index, test_index in ss.split(traindata1,y1):
  #print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = traindata1.iloc[train_index], traindata1.iloc[test_index]
  y_train, y_test = y1.iloc[train_index], y1.iloc[test_index]


X_train.shape,X_test.shape

((1154, 219), (129, 219))

In [ ]:
y_train.value_counts()

1    911
0    243
Name: KAN, dtype: int64

In [ ]:
## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.01,0.03,0.05, 0.10,0.12, 0.15,0.18, 0.20, 0.25, 0.30,0.35,0.4] ,
 "max_depth"        : [ 6,8, 10, 12,14, 15,16],
 "gamma"            : [ 0.0,0.2,0.4 ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350,370,400]   
}
params1 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9,2,2.2,2.4,2.6,2.8,3,3.2,3.4],
   'kernel' : ['rbf'],
   'degree':[1,2,3,4,5,6,7,8],
    'class_weight':['balanced',''],
  'decision_function_shape':['ovo', 'ovr']
}
params2 = {
    'C' :[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,1,1,2,1,3,1,5,1.6,1.7,1.9,2,2.2],
    'class_weight':['balanced',''],
    'solver':['newton-cg','lbfgs', 'liblinear', 'sag', 'saga'],
    'max_iter':[20,40,60,80,100,120,140,180,200,230,250,270,300]
}
params3 = {
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_samples_split" : [ 1,2, 3,4, 5,6, 7 ],
 "max_features" : [ 'auto','sqrt','log2' ],
 'n_estimators'     : [50,60,70,80,85,90,100,110,120,150,170,190,200,250,300,350]   
}

In [ ]:
classifier= XGBClassifier()
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=25,scoring='roc_auc'
                                 ,n_jobs=-1,cv=2,verbose=3)

grid_search=GridSearchCV(classifier,param_grid=params,scoring='roc_auc',
                                 n_jobs=-1,cv=2,verbose=3)

In [ ]:
grid_search.fit(X_train,y_train)

In [ ]:
print(grid_search.best_estimator_)
grid_search.best_score_

In [ ]:
lr = LogisticRegression(C=0.1, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=60, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

y_pred = lr.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8604651162790697
[[16 11]
 [ 7 95]]
              precision    recall  f1-score   support

           0       0.70      0.59      0.64        27
           1       0.90      0.93      0.91       102

    accuracy                           0.86       129
   macro avg       0.80      0.76      0.78       129
weighted avg       0.85      0.86      0.86       129



In [ ]:
svm = SVC(gamma='scale',C=5,kernel='rbf',class_weight='',probability=True,
          decision_function_shape='ovo',degree=1)

svm.fit(X_train, y_train)
print(svm.score(X_test, y_test))

y_pred = svm.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8837209302325582
[[ 13  14]
 [  1 101]]
              precision    recall  f1-score   support

           0       0.93      0.48      0.63        27
           1       0.88      0.99      0.93       102

    accuracy                           0.88       129
   macro avg       0.90      0.74      0.78       129
weighted avg       0.89      0.88      0.87       129



In [ ]:
rf = RandomForestClassifier(n_estimators=130,max_depth=4)
rf.fit(X_train, y_train)
print(rf.score(X_test, y_test))

y_pred = rf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.7984496124031008
[[  1  26]
 [  0 102]]
              precision    recall  f1-score   support

           0       1.00      0.04      0.07        27
           1       0.80      1.00      0.89       102

    accuracy                           0.80       129
   macro avg       0.90      0.52      0.48       129
weighted avg       0.84      0.80      0.72       129



In [ ]:
xf = XGBClassifier(learning_rate=0.05, max_depth=15,
              n_estimators=140)

xf.fit(X_train, y_train)


print(xf.score(X_test, y_test))
y_pred = xf.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.8837209302325582
[[ 13  14]
 [  1 101]]
              precision    recall  f1-score   support

           0       0.93      0.48      0.63        27
           1       0.88      0.99      0.93       102

    accuracy                           0.88       129
   macro avg       0.90      0.74      0.78       129
weighted avg       0.89      0.88      0.87       129



In [ ]:

score=cross_val_score(xf,traindata1,y1,cv=2,scoring='roc_auc')
print(score)
score.mean()

[0.8278764  0.89239496]


0.8601356806206304

In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.utils import class_weight
from keras import optimizers
tf.__version__


'2.3.0'

In [ ]:
X_train.shape

(1154, 219)

In [ ]:
!rm -r ./model/
!rm ./model.zip

In [ ]:
model = tf.keras.models.Sequential([
tf.keras.layers.InputLayer(input_shape=219),
tf.keras.layers.Dense(112,activation='relu'),
#tf.keras.layers.Dropout(0.05),
#tf.keras.layers.Dense(32,activation='relu'),
#tf.keras.layers.Dense(8,activation='relu'),
tf.keras.layers.Dense(4,activation='relu'),
tf.keras.layers.Dense(1,activation='sigmoid'),
])


In [ ]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(),
              )
model.summary()
weights = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(weights)
temp = {0:weights[0],1:weights[1]}
type(temp)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 112)               24640     
_________________________________________________________________
dense_67 (Dense)             (None, 4)                 452       
_________________________________________________________________
dense_68 (Dense)             (None, 1)                 5         
Total params: 25,097
Trainable params: 25,097
Non-trainable params: 0
_________________________________________________________________
[2.3744856  0.63336992]


dict

In [ ]:
#!unzip model_KAN.zip
#!rm model_KAN.zip
model = tf.keras.models.load_model('model_KAN/')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 96)                21120     
_________________________________________________________________
dense_4 (Dense)              (None, 4)                 388       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 5         
Total params: 21,513
Trainable params: 21,513
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train,y_train,epochs=1,validation_data=(X_test,y_test))

37/37 [==============================] - 0s 4ms/step - loss: 0.2469 - val_loss: 0.3854


In [ ]:
model.save('model_KAN')
!zip -r model_KAN.zip ./model_KAN

INFO:tensorflow:Assets written to: model_KAN/assets
  adding: model_KAN/ (stored 0%)
  adding: model_KAN/saved_model.pb (deflated 88%)
  adding: model_KAN/variables/ (stored 0%)
  adding: model_KAN/variables/variables.index (deflated 61%)
  adding: model_KAN/variables/variables.data-00000-of-00001 (deflated 15%)
  adding: model_KAN/assets/ (stored 0%)


In [ ]:
preds = model.predict(X_test)
preds1 = np.where(preds>0.5,1,0)
preds2 = np.where(model.predict(X_train)>0.5,1,0)
print(np.unique(preds1,return_counts=True))
print(accuracy_score(y_train,preds2))
print(accuracy_score(y_test,preds1))
print(confusion_matrix(y_test,preds1))
print(roc_auc_score(y_test,preds))

(array([0, 1]), array([ 14, 115]))
0.9116117850953206
0.8837209302325582
[[ 13  14]
 [  1 101]]
0.8311546840958606


In [ ]:
X_testdata = pd.read_csv('X_testData_1.csv')
Y_testdata_nolabel = pd.read_csv('Y_testData_1_nolabels_KAN.csv')
output_id = set(Y_testdata_nolabel['ID'])
len(output_id)

498

In [ ]:
X_testdata.drop('SNP_CN_2714366_C967A_V323L_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713795_C329T_inter_Rv2415c_eis',axis=1,inplace=True)
X_testdata.drop('SNP_I_2713872_C252A_inter_Rv2415c_eis',axis=1,inplace=True)

generate_id = X_testdata['ID']
X_testdata.drop('ID',axis=1,inplace=True)
#generate_id

In [ ]:
#X_testdata = X_testdata.drop(columns=correlation.index,axis=1)
#Use following code to generate output for ML models
output_pred = rf.predict_proba(X_testdata)
out = pd.DataFrame({'KAN':output_pred[:,1]})
out['ID'] = generate_id
out = out[['ID','KAN']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
print(out['KAN'].value_counts())
#sorted(out['KAN'])[:10]

In [ ]:
#X_testdata = X_testdata.drop(columns=correlation.index,axis=1)
#Use following code to generate output for Neural Network model
output_pred = model.predict(X_testdata)
out = pd.DataFrame({'KAN':output_pred[:,0]})
out['ID'] = generate_id
out = out[['ID','KAN']]
#out
for ind in out.index:
    if out['ID'][ind] not in output_id:
        out.drop(ind,inplace=True)
print(out['KAN'].value_counts())
sorted(out['KAN'])[:10]

In [ ]:
out.to_csv('Group_7_output.csv',index=None)